In [9]:
import pandas as pd
from tqdm import tqdm
from sqlmodel import Session
import yaml 
import os


In [10]:
with open("./comfigmap.yaml","r") as f:
    config = yaml.safe_load(f)

In [11]:
config

{'data_path': '/Users/jonathanbizet/Documents/Formation_IA/MLOPS/projet_reco_films/',
 'postgres_params': {'user': 'postgres',
  'password': 'recommendation_films_oct_23_MLOPS',
  'host_url': 'reco-films-db.ck2uuvj8tg5b.eu-west-3.rds.amazonaws.com',
  'base': 'postgres'},
 'sqlite': {'path': '/Users/jonathanbizet/Documents/Formation_IA/MLOPS/projet_reco_films/reco_films.db'},
 'database': 'sqlite'}

# Chargement des donnée dans un dataframe

In [12]:
genome_tags_csv = pd.read_csv(os.path.join(config["data_path"],"ml-20m/genome-tags.csv"))
genome_score_csv = pd.read_csv(os.path.join(config["data_path"],"ml-20m/genome-scores.csv"))
tags_csv = pd.read_csv(os.path.join(config["data_path"],"ml-20m/tags.csv"))
links_imdb_csv = pd.read_csv(os.path.join(config["data_path"],"ml-20m/links.csv"))
ratings_csv = pd.read_csv(os.path.join(config["data_path"],"ml-20m/ratings.csv"))
movies_csv = pd.read_csv(os.path.join(config["data_path"],"ml-20m/movies.csv"))

## Supression de films en double

In [13]:
movies_csv = movies_csv.merge(right=links_imdb_csv,how="left")

In [14]:
movies_csv [movies_csv[['title',"imdbId"]].duplicated(keep = False)]

,movieId,title,genres,imdbId,tmdbId


# Création des modèles et des tables

In [15]:
from typing import Optional

from sqlmodel import Field, SQLModel, create_engine


class Genre(SQLModel, table=True):
    __table_args__ = {'extend_existing': True}
    genreid: Optional[int] = Field(default=None,primary_key=True)
    name: str

class Movie (SQLModel, table=True):
    __table_args__ = {'extend_existing': True}
    movieid: Optional[int] = Field(default=None,primary_key=True,index=True)
    title: str
    imdbid: Optional[int]
    tmdbid: Optional[int]

class Movie_Genre(SQLModel, table=True):
    __table_args__ = {'extend_existing': True}
    movieid: int = Field(default=None, foreign_key="movie.movieid",primary_key=True,index=True)
    genreid: int = Field(default=None, foreign_key="genre.genreid",primary_key=True)
    


class Tag(SQLModel, table=True):
    __table_args__ = {'extend_existing': True}
    tagid : int = Field(default=None, primary_key=True)
    userid : int 
    movieid : int = Field(default=None, foreign_key="movie.movieid",index=True)
    tag : str
    timestamp : int

class Genome_Tag(SQLModel, table=True):
    __table_args__ = {'extend_existing': True}
    gtagid : int = Field(default=None,primary_key=True)
    tag: str
    
class Genome_Score(SQLModel, table=True, extend_existing=True):
    __table_args__ = {'extend_existing': True}
    movieid: int = Field(default=None, foreign_key="movie.movieid", primary_key=True,index=True)
    gtagid: int = Field(default=None, foreign_key="genome_tag.gtagid",primary_key=True)
    relevance: float = Field(ge=0, le=1)

    
class Rating (SQLModel, table=True):
    __table_args__ = {'extend_existing': True}
    userid: int = Field(default=None,primary_key=True,index=True)
    movieid: int = Field(default=None, foreign_key="movie.movieid",primary_key=True,index=True)
    rating:float
    timestamp: int
    

# Connection à la base de donnée

In [16]:
if config["database"] == 'postgres':
    user = config["postgres_params"]["user"]
    password = config["postgres_params"]["password"]
    host_url = config["postgres_params"]["host_url"]
    base = config["postgres_params"]["base"]

    database_url = f"postgresql://{user}:{password}@{host_url}:5432/{base}"
else:
    database_url = f"sqlite:///{config['sqlite']['path']}"

engine = create_engine(database_url, echo=True)

SQLModel.metadata.create_all(engine)

2023-12-18 20:03:27,824 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-12-18 20:03:27,824 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("genre")
2023-12-18 20:03:27,824 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-12-18 20:03:27,825 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("movie")
2023-12-18 20:03:27,825 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-12-18 20:03:27,825 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("movie_genre")
2023-12-18 20:03:27,825 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-12-18 20:03:27,826 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("tag")
2023-12-18 20:03:27,826 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-12-18 20:03:27,826 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("tag")
2023-12-18 20:03:27,826 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-12-18 20:03:27,826 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("genome_tag")
2023-12-18 20:03:27,827 INFO sqlalchemy.engine.Engine [raw sql] ()
2023

In [17]:
database_url

'sqlite:////Users/jonathanbizet/Documents/Formation_IA/MLOPS/projet_reco_films/reco_films.db'

In [10]:
database_url

'sqlite:////Users/jonathanbizet/Documents/Formation_IA/MLOPS/projet_reco_films/reco_films.db'

# Insertion des Genres

In [11]:
list_genres_movies = movies_csv.genres.apply(lambda x: x.split('|'))
list_genre = set().union(*list_genres_movies)
list_genre = sorted(list(list_genre))
dict_genre = dict([(genre,idgenre) for idgenre ,genre in enumerate(list_genre,1) ])

In [12]:
from sqlmodel import Session
with Session(engine) as session: 
    for idgenre ,genre in enumerate(list_genre,1):
       session.add( Genre(genreid=idgenre,name=genre))
    session.commit()
    

2023-12-18 19:59:29,586 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-12-18 19:59:29,587 INFO sqlalchemy.engine.Engine INSERT INTO genre (genreid, name) VALUES (?, ?)
2023-12-18 19:59:29,587 INFO sqlalchemy.engine.Engine [generated in 0.00024s] ((1, '(no genres listed)'), (2, 'Action'), (3, 'Adventure'), (4, 'Animation'), (5, 'Children'), (6, 'Comedy'), (7, 'Crime'), (8, 'Documentary')  ... displaying 10 of 20 total bound parameter sets ...  (19, 'War'), (20, 'Western'))
2023-12-18 19:59:29,588 INFO sqlalchemy.engine.Engine ROLLBACK


IntegrityError: (sqlite3.IntegrityError) UNIQUE constraint failed: genre.genreid
[SQL: INSERT INTO genre (genreid, name) VALUES (?, ?)]
[parameters: ((1, '(no genres listed)'), (2, 'Action'), (3, 'Adventure'), (4, 'Animation'), (5, 'Children'), (6, 'Comedy'), (7, 'Crime'), (8, 'Documentary')  ... displaying 10 of 20 total bound parameter sets ...  (19, 'War'), (20, 'Western'))]
(Background on this error at: https://sqlalche.me/e/14/gkpj)

# Insertion des Films (Movies)

In [ ]:
l_movies = list()
for index, mov in movies_csv.iterrows():
    
    mv = Movie(
        **{
            "movieid": mov["movieId"],
            "title": mov["title"],
            "imdbid": mov["imdbId"],
            "tmdbid":  mov["tmdbId"]
        }
    )
    l_movies.append(mv)

In [ ]:
with Session(engine) as session: 
    session.add_all(l_movies)
    session.commit()  

2023-12-18 18:37:36,750 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-12-18 18:37:36,846 INFO sqlalchemy.engine.Engine INSERT INTO movie (movieid, title, imdbid, tmdbid) VALUES (?, ?, ?, ?)
2023-12-18 18:37:36,847 INFO sqlalchemy.engine.Engine [generated in 0.02059s] ((1, 'Toy Story (1995)', 114709, 862), (2, 'Jumanji (1995)', 113497, 8844), (3, 'Grumpier Old Men (1995)', 113228, 15602), (4, 'Waiting to Exhale (1995)', 114885, 31357), (5, 'Father of the Bride Part II (1995)', 113041, 11862), (6, 'Heat (1995)', 113277, 949), (7, 'Sabrina (1995)', 114319, 11860), (8, 'Tom and Huck (1995)', 112302, 45325)  ... displaying 10 of 27278 total bound parameter sets ...  (131260, 'Rentun Ruusu (2001)', 249110, 32099), (131262, 'Innocence (2014)', 1724965, 286971))
2023-12-18 18:37:36,939 INFO sqlalchemy.engine.Engine COMMIT


# Insertion de Movie Genre

In [ ]:
l_movie_genre = list()
for index, movie in movies_csv.iterrows():
    for genre in dict_genre:
        if genre in movie["genres"]:
            mg = Movie_Genre(movieid=movie['movieId'],genreid=dict_genre[genre], )
            l_movie_genre.append(mg)

In [ ]:
l_movie_genre[:4]

[Movie_Genre(movieid=1, genreid=3),
 Movie_Genre(movieid=1, genreid=4),
 Movie_Genre(movieid=1, genreid=5),
 Movie_Genre(movieid=1, genreid=6)]

In [ ]:


with Session(engine) as session:
    session.add_all(l_movie_genre)
    session.commit()

2023-12-18 18:37:39,289 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-12-18 18:37:39,609 INFO sqlalchemy.engine.Engine INSERT INTO movie_genre (movieid, genreid) VALUES (?, ?)
2023-12-18 18:37:39,610 INFO sqlalchemy.engine.Engine [generated in 0.17715s] ((1, 3), (1, 4), (1, 5), (1, 6), (1, 10), (2, 3), (2, 5), (2, 10)  ... displaying 10 of 54406 total bound parameter sets ...  (131262, 10), (131262, 12))
2023-12-18 18:37:39,823 INFO sqlalchemy.engine.Engine COMMIT


# Donnée Genome_tag

In [ ]:
l_genome_tag = list()
for index, gt in genome_tags_csv.iterrows():
    l_genome_tag.append(Genome_Tag(tag=gt['tag']))

In [ ]:
l_genome_tag[:4]

[Genome_Tag(gtagid=None, tag='007'),
 Genome_Tag(gtagid=None, tag='007 (series)'),
 Genome_Tag(gtagid=None, tag='18th century'),
 Genome_Tag(gtagid=None, tag='1920s')]

In [ ]:
with Session(engine) as session:
    session.add_all(l_genome_tag)
    session.commit()

# Données Genome_Score

In [ ]:
genome_score_csv.head()

,movieId,tagId,relevance
0,1,1,0.02500
1,1,2,0.02500
2,1,3,0.05775
3,1,4,0.09675
4,1,5,0.14675


In [ ]:

l_genome_score = list()
for index, gs in genome_score_csv.iterrows():

    l_genome_score.append(Genome_Score(movieid=gs['movieId'],gtagid=gs['tagId'],relevance=gs['relevance']))
    if len(l_genome_score)%100000 == 0:
        with Session(engine) as session:
            session.add_all(l_genome_score)
            session.commit()
        l_genome_score = list()
with Session(engine) as session:
    print(len(l_genome_score))
    session.add_all(l_genome_score)
    session.commit()


# Données Rating 

In [ ]:
ratings_csv.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,1112486027
1,1,29,3.5,1112484676
2,1,32,3.5,1112484819
3,1,47,3.5,1112484727
4,1,50,3.5,1112484580


In [ ]:

l_rating = list()
for index, rate in ratings_csv.iterrows():

    l_rating.append(Rating(movieid=rate['movieId'],userid=rate['userId'],rating=rate['rating'],timestamp=rate["timestamp"]))
    if len(l_rating)%100000 == 0:
        with Session(engine) as session:
            session.add_all(l_rating)
            session.commit()
        l_rating = list()
with Session(engine) as session:
    print(len(l_rating))
    session.add_all(l_rating)
    session.commit()


# Données Tag

In [ ]:
tags_csv

,userId,movieId,tag,timestamp
0,18,4141,Mark Waters,1240597180
1,65,208,dark hero,1368150078
2,65,353,dark hero,1368150079
3,65,521,noir thriller,1368149983
4,65,592,dark hero,1368150078
...,...,...,...,...
465559,138446,55999,dragged,1358983772
465560,138446,55999,Jason Bateman,1358983778
465561,138446,55999,quirky,1358983778
465562,138446,55999,sad,1358983772


In [18]:
l_tags = list()
for index, tag_ in tags_csv.iterrows():
    l_tags.append(Tag(movieid=tag_['movieId'],userid=tag_['userId'],tag=tag_['tag'],timestamp=tag_["timestamp"]))
    if len(l_tags)%100000 == 0:
        with Session(engine) as session:
            session.add_all(l_tags)
            session.commit()
        l_tags = list()
with Session(engine) as session:
    session.add_all(l_tags)
    session.commit()